In [1]:
%pip install --upgrade datasets
%pip install --upgrade transformers
%pip install --upgrade optuna

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 74.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datasets import Dataset,load_dataset
import ast
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support
from torch.utils.data import DataLoader
from transformers import default_data_collator,set_seed
import torch
import optuna
import shutil
seed = 42
set_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
import time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    # Get the number of GPUs available
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")
    
    # Print out the name of each GPU and memory details
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"Total memory: {torch.cuda.get_device_properties(i).total_memory / 1e9} GB")
    
    # If multiple GPUs are available, use DataParallel for multi-GPU
    if num_gpus > 1:
        print("Using DataParallel for multi-GPU training.")
        # Example: Wrap your model with DataParallel
    else:
        print("Only one GPU available, using single GPU mode.")
else:
    print("No GPUs available, using CPU.")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/python/NLBSE_Dataset_Python.csv
Number of GPUs available: 1
GPU 0: Tesla P100-PCIE-16GB
Total memory: 17.059545088 GB
Only one GPU available, using single GPU mode.


In [3]:
df = pd.read_csv("/kaggle/input/python/NLBSE_Dataset_Python.csv")
test = load_dataset('NLBSE/nlbse25-code-comment-classification')['python_test']
df.info()

README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

java_train-00000-of-00001.parquet:   0%|          | 0.00/680k [00:00<?, ?B/s]

java_test-00000-of-00001.parquet:   0%|          | 0.00/174k [00:00<?, ?B/s]

python_train-00000-of-00001.parquet:   0%|          | 0.00/126k [00:00<?, ?B/s]

python_test-00000-of-00001.parquet:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

...

Generating python_train split:   0%|          | 0/1884 [00:00<?, ? examples/s]

Generating python_test split:   0%|          | 0/406 [00:00<?, ? examples/s]

Generating pharo_train split:   0%|          | 0/1298 [00:00<?, ? examples/s]

Generating pharo_test split:   0%|          | 0/289 [00:00<?, ? examples/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10170 entries, 0 to 10169
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   class             10170 non-null  object
 1   comment_sentence  10169 non-null  object
 2   labels            10170 non-null  object
 3   types             10170 non-null  object
dtypes: object(4)
memory usage: 317.9+ KB


In [4]:
df.head(10)
# if we are in a kaggle environment we need to use that
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Wandb")
import wandb
# Replace YOUR_API_KEY with your actual API key
wandb.login(key=secret_value_0)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
df.drop_duplicates(subset=['comment_sentence'], keep='first', inplace=True)
df.info()
null_rows = df[df['comment_sentence'].isnull()]

print("Rows with null values in 'comment_sentence':")
print(null_rows)

<class 'pandas.core.frame.DataFrame'>
Index: 8176 entries, 0 to 10168
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   class             8176 non-null   object
 1   comment_sentence  8175 non-null   object
 2   labels            8176 non-null   object
 3   types             8176 non-null   object
dtypes: object(4)
memory usage: 319.4+ KB
Rows with null values in 'comment_sentence':
               class comment_sentence       labels types
9172  MultiValueDict              NaN  [1 0 0 0 0]    ru


In [6]:
df_cleaned = df.dropna(subset=['comment_sentence'])
print("DataFrame shape after removing nulls:", df_cleaned.shape)
df_cleaned.info()

DataFrame shape after removing nulls: (8175, 4)
<class 'pandas.core.frame.DataFrame'>
Index: 8175 entries, 0 to 10168
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   class             8175 non-null   object
 1   comment_sentence  8175 non-null   object
 2   labels            8175 non-null   object
 3   types             8175 non-null   object
dtypes: object(4)
memory usage: 319.3+ KB


In [7]:
pattern = r'https?://\S+|\t'
rows_with_pattern = df_cleaned.apply(lambda row: row.astype(str).str.contains(pattern).any(), axis=1)

# Count rows with patterns
num_rows_with_pattern = rows_with_pattern.sum()
print(f"\nNumber of rows containing patterns: {num_rows_with_pattern}")

# Remove `//` or `*` from all columns
df_cleaned = df_cleaned.replace(pattern, '', regex=True)


Number of rows containing patterns: 4


In [8]:
df = df_cleaned
df['combo'] = df['comment_sentence'] +"  |  "+  df['class']
python_dataset = Dataset.from_pandas(df)
# Split the dataset into train and validation subsets
train_test_split = python_dataset.train_test_split(test_size=0.2, seed=42)

# Extract train and validation datasets
python_train = train_test_split['train']
python_test = train_test_split['test']
python_labels = ['Usage', 'Parameters', 'DevelopmentNotes', 'Expand', 'Summary']
data_collator = default_data_collator
num_labels = len(python_labels)


In [9]:
# Initialize global variables
best_loss = float('inf')  # Tracks the best evaluation loss
best_model_path = "./best_model"  # Directory to store the best model

# Ensure best_model directory does not exist at the start
if os.path.exists(best_model_path):
    shutil.rmtree(best_model_path)

# Define a function to clear a directory's contents
def clear_directory(directory):
    """Removes all contents inside a directory."""
    if not os.path.exists(directory):
        os.makedirs(directory)
        return
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # Remove file or symbolic link
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Remove directory
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

# Define the evaluation function
def evaluate(new_dataset, model, tokenizer, labels, batch_size=16, device='cuda'):
    # Move model to device
    model.to(device)

    # Prepare data
    texts = new_dataset['combo']
    true_labels = np.array(new_dataset['labels'])

    # Tokenize the inputs
    inputs = tokenizer(texts, truncation=True, padding=True, return_tensors="pt")
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Split into batches
    num_batches = (len(texts) + batch_size - 1) // batch_size
    predictions = []
    start_time = time.time()

    # Perform inference in batches
    for i in range(num_batches):
        batch_input_ids = input_ids[i * batch_size: (i + 1) * batch_size]
        batch_attention_mask = attention_mask[i * batch_size: (i + 1) * batch_size]

        with torch.no_grad():
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
            logits = outputs.logits
            preds = (logits.sigmoid() > 0.5).int().cpu().numpy()
            predictions.append(preds)

    end_time = time.time()
    avg_runtime = (end_time - start_time) / num_batches

    # Concatenate predictions
    predictions = np.vstack(predictions)

    # Evaluate metrics for each label
    metrics = []
    for i, label in enumerate(labels):
        tp = np.sum((true_labels[:, i] == 1) & (predictions[:, i] == 1))
        fp = np.sum((true_labels[:, i] == 0) & (predictions[:, i] == 1))
        fn = np.sum((true_labels[:, i] == 1) & (predictions[:, i] == 0))
        tn = np.sum((true_labels[:, i] == 0) & (predictions[:, i] == 0))

        precision = tp / (tp + fp) if tp + fp > 0 else 0
        recall = tp / (tp + fn) if tp + fn > 0 else 0
        f1 = (2 * precision * recall) / (precision + recall) if precision + recall > 0 else 0

        metrics.append({'label': label, 'precision': precision, 'recall': recall, 'f1': f1})

    # Convert metrics to DataFrame
    metrics_df = pd.DataFrame(metrics)
    average_f1 = metrics_df['f1'].mean()
    print("Average F1 on test set:", average_f1)
    return metrics_df, avg_runtime

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlnet/xlnet-base-cased")

# Define the model initialization function
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        "xlnet/xlnet-base-cased",
        num_labels=num_labels,
        problem_type="multi_label_classification",
    )

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["combo"], truncation=True, padding="max_length", max_length=128)

tokenized_train = python_train.map(tokenize_function, batched=True)
tokenized_test = python_test.map(tokenize_function, batched=True)

# Convert labels to tensors
def encode_labels(examples):
    if isinstance(examples['labels'], str):
        examples["labels"] = examples["labels"].replace(" ", ",")
        labels = ast.literal_eval(examples['labels'])
    else:
        labels = examples['labels']
    # Convert labels to tensors
    labels = torch.tensor(labels, dtype=torch.float32)
    return {'labels': labels}

tokenized_train = tokenized_train.map(encode_labels)
tokenized_test = tokenized_test.map(encode_labels)

# Format datasets for PyTorch
tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Define compute_metrics function for Trainer
def compute_metrics(pred):
    logits, labels = pred
    print(f"Logits Shape: {logits.shape}, Labels Shape: {labels.shape}")
    # Apply sigmoid to logits to convert to probabilities
    probs = 1 / (1 + np.exp(-logits))  # Sigmoid function
    preds = (probs > 0.5).astype(int)  # Threshold for multi-label classification
    # Compute metrics
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="micro")
    if f1 == 0:
        print("F1 score is zero. Resetting the trial.")
        raise optuna.exceptions.TrialPruned()  # Prune the trial to reset it
    return {"precision": precision, "recall": recall, "f1": f1}

# Define the Optuna objective function
def optuna_objective(trial):
    global best_loss, best_model_path

    # Define hyperparameter search space
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-4)
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.1)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16, 32])

    # Define unique temporary directories for each trial
    temp_output_dir = f"./temp_results_bert_base_trial_{trial.number}"
    temp_logs_dir = f"./temp_logs_bert_base_trial_{trial.number}"

    # Create temporary directories
    os.makedirs(temp_output_dir, exist_ok=True)
    os.makedirs(temp_logs_dir, exist_ok=True)

    # Define TrainingArguments
    training_args = TrainingArguments(
        output_dir=temp_output_dir,  # Unique temporary directory for each trial
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=12,
        weight_decay=weight_decay,
        logging_dir=temp_logs_dir,
        logging_steps=100,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        save_total_limit=1,  # Keep only the best checkpoint
    )

    # Initialize Trainer
    trainer = Trainer(
        model_init=model_init,  # Ensures a fresh model for each trial
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_test,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        data_collator=default_data_collator,
    )

    # Perform training
    trainer.train()

    # Evaluate on the validation set
    eval_results = trainer.evaluate()
    eval_loss = eval_results["eval_loss"]
    print(f"Trial {trial.number} - Evaluation Loss: {eval_loss}")

    # Check if this trial has the best loss so far
    if eval_loss < best_loss:
        best_loss = eval_loss
        print(f"Trial {trial.number} achieved the new best loss: {best_loss}. Saving the model.")

        # Remove existing best_model directory if it exists
        if os.path.exists(best_model_path):
            shutil.rmtree(best_model_path)

        # **Save the best model correctly using trainer.save_model**
        trainer.save_model(best_model_path)

    else:
        print(f"Trial {trial.number} did not beat the best loss: {best_loss}.")

    # Clean up the temporary directories to save space
    shutil.rmtree(temp_output_dir)
    shutil.rmtree(temp_logs_dir)

    return eval_loss

# Run Optuna search
study = optuna.create_study(direction="minimize")
study.optimize(optuna_objective, n_trials=16)

# Display best hyperparameters
print("Best Hyperparameters:", study.best_params)
print("Best Evaluation Loss:", study.best_value)

best_model = AutoModelForSequenceClassification.from_pretrained(best_model_path)
best_tokenizer = AutoTokenizer.from_pretrained("xlnet/xlnet-base-cased")

# (Optional) Evaluate the loaded best model
metrics_df, avg_runtime = evaluate(
    new_dataset=test,
    model=best_model,
    tokenizer=best_tokenizer,
    labels=python_labels,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)
print("Metrics of the best model:")
print(metrics_df)




config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

Map:   0%|          | 0/6540 [00:00<?, ? examples/s]

Map:   0%|          | 0/1635 [00:00<?, ? examples/s]

...

Logits Shape: (1635, 5), Labels Shape: (1635, 5)
Trial 15 - Evaluation Loss: 0.062307607382535934
Trial 15 did not beat the best loss: 0.06218144670128822.


[I 2024-12-08 21:20:55,860] Trial 15 finished with value: 0.062307607382535934 and parameters: {'learning_rate': 6.381748041800216e-05, 'weight_decay': 0.06917221802020096, 'batch_size': 32}. Best is trial 12 with value: 0.06218144670128822.


Best Hyperparameters: {'learning_rate': 2.8563875469636118e-05, 'weight_decay': 0.09823387650870327, 'batch_size': 32}
Best Evaluation Loss: 0.06218144670128822


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Average F1 on test set: 0.6366014684221979
Metrics of the best model:
              label  precision    recall        f1
0             Usage   0.807339  0.727273  0.765217
1        Parameters   0.800000  0.843750  0.821293
2  DevelopmentNotes   0.411765  0.341463  0.373333
3            Expand   0.472222  0.531250  0.500000
4           Summary   0.673684  0.780488  0.723164
